# Bias-adjustment comparison

Explanation...

### Setup

In [ ]:
print(eg_storage.keys())

In [ ]:
# import what we need for the statistical/visual comparison
from statsmodels.distributions.empirical_distribution import ECDF
import pandas as pd

### Load Crossing Data

In [ ]:
# define the loadpath for the crossing years of a particular threshold value
deltaC=1.07
cross_loadpath='./data/analysis/CMIP5_crossing_year_'+str(np.round(deltaC,2))+'C.csv'
# load the file from the save file for each model, reindex it with model names
cross_dat=pd.read_csv(cross_loadpath,index_col=[0])

# print the results for the example model
print(cross_dat.loc[mname])

## Mapping the example bias adjustments

In [ ]:
# calculate the specific bias adjustment magnitude by differencing raw and BA
BA_magnitude_forced=eg_storage['forced_xsimadj_BA']-eg_storage['forced_xsimadj_raw']
BA_magnitude_natural=eg_storage['natural_xsimadj_BA']-eg_storage['natural_xsimadj_raw']

# wrap around the prime meridian before plotting
BA_forced_cyc = util.add_cyclic_point(BA_magnitude_forced.TMAX, 'lon')
BA_natural_cyc = util.add_cyclic_point(BA_magnitude_natural.TMAX, 'lon')

In [ ]:
stop_here

In [ ]:
fig=plt.figure()



# plot the result
ax = plt.subplot(6, 4, hh+1, projection=ccrs.PlateCarree())

levs=np.arange(-10,10,1)
datp=ax.contourf(DIFF_qj_cyc.lon,DIFF_qj_cyc.lat,DIFF_qj_cyc, \
                        levels=levs, transform=ccrs.PlateCarree(), \
                        cmap='RdBu_r', extend='both')

# draw the coastlines
ax.coastlines()
ax.gridlines(alpha=0.25,linestyle=(0, (5, 1)) )

# add the model string
plt.text(-165,-53,mname,color='black',fontsize=14,transform=ccrs.PlateCarree())

# clean up
del ba_qj, un_qj, diff_qj, un_dat, ba_dat
del DIFF_qj_cyc, zname
print(mname)

plt.show()